In [10]:
import pandas as pd
import time
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import nltk

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk import word_tokenize

In [16]:
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/achandrasekhar/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/achandrasekhar/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/achandrasekhar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [17]:
tweets = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [18]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [19]:
tweet=tweets[['text','target']]
tweet

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...
7608,Two giant cranes holding a bridge collapse int...,1
7609,@aria_ahrary @TheTawniest The out of control w...,1
7610,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,Police investigating after an e-bike collided ...,1


In [20]:
tweet.isnull()

,text,target
0,False,False
1,False,False
2,False,False
3,False,False
4,False,False
...,...,...
7608,False,False
7609,False,False
7610,False,False
7611,False,False


In [21]:
def stemmer(text):
    stemm = PorterStemmer()
    return ' '.join([stemm.stem(w) for w in word_tokenize(text)])

In [22]:
def lemmetizer(text):
    lemmetizer=WordNetLemmatizer()
    return ' '.join([lemmetizer.lemmatize(w) for w in word_tokenize(text)])

In [23]:
def print_model_data(name,time,model,X_test,y_test,is_grid,title='Confusion Matrix',cmap=plt.cm.Blues):
    print('###################################################################################################################')
    print(name+' fit time in seconds----'+str(time))
    print('###################################################################################################################')
    score = model.score(X_test,y_test)
    print('accuracy of '+name+'----'+str(score))
    print('###################################################################################################################')
    model_param = model.get_params()
    print('parameters of model '+name+'----'+str(params))
    print('###################################################################################################################')
    
    pred=model.predict(X_test)
    accuracy = accuracy_score(y_test, pred)
    print('parameters of model '+name+'----'+str(accuracy))
    print('###################################################################################################################')
    auc = roc_auc_score(y_test,pred)
    print('parameters of model '+name+'----'+str(auc))
    print('###################################################################################################################')
    
    conf = metrics.ConfusionMatrixDisplay.from_predictions(y_test, pred)
    plt.title(title)
    plt.ylabel('True label')
    plt.xlabel('Predicated label')
    if(is_grid==True):
        best_params=model.best_params_
        comparison.loc[len(comparison.index)] = [name,best_params,score,time,accuracy,auc]
    else:
        comparison.loc[len(comparison.index)] = [name,model_param,score,time,accuracy,auc]
    
    print(comparison)

In [24]:
def model_execute(model,X_train,y_train):
    start=time.time()
    model.fit(X_train,y_train)
    end=time.time()
    return end-start